# 🏆 RIAWELC - Kaynak Hatası Tespit Sistemi

**ResNet18 Model Test Arayüzü (%100 Accuracy)**

Transfer Learning ile eğitilmiş ResNet18 modelini kullanarak kaynak hatalarını tespit eder.

---

## 🎯 Model Performansı:
- **Test Accuracy:** %100.00 🏆
- **Validation Accuracy:** %99.97
- **Test Loss:** 0.0002
- **Toplam Test:** 2,443 görüntü - HEPSİ DOĞRU!

## 🔍 Tespit Edilen Hatalar:
- 🔴 **Difetto1:** Çatlak (Crack)
- 🟡 **Difetto2:** Nüfuziyet Eksikliği (Lack of Penetration)
- 🟢 **Difetto4:** Gözeneklilik (Porosity)
- ⚪ **NoDifetto:** Hatasız (No Defect)

---

**Geliştirici:** [Emine BALYAN ÇELİKKAYA]

In [1]:
import subprocess
import sys

print("🔧 Gerekli paketler güncelleniyor...")

# typing_extensions güncelle
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "typing_extensions"])
print("✅ typing_extensions güncellendi")

# Gradio kur
subprocess.check_call([sys.executable, "-m", "pip", "install", "gradio"])
print("✅ Gradio kuruldu")

print("\n⚠️ ÖNEMLİ: Jupyter Kernel'i yeniden başlat!")
print("📌 Kernel → Restart Kernel")
print("📌 Sonra tekrar çalıştır!")

🔧 Gerekli paketler güncelleniyor...
✅ typing_extensions güncellendi
✅ Gradio kuruldu

⚠️ ÖNEMLİ: Jupyter Kernel'i yeniden başlat!
📌 Kernel → Restart Kernel
📌 Sonra tekrar çalıştır!


In [2]:
# Kütüphaneleri yükle
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import numpy as np
import gradio as gr

print("✅ Tüm kütüphaneler yüklendi!")

# GPU kontrol
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Kullanılan cihaz: {device}")

# Sınıf isimleri
class_names = {
    0: 'Difetto1 - Çatlak (Crack)',
    1: 'Difetto2 - Nüfuziyet Eksikliği (Lack of Penetration)',
    2: 'Difetto4 - Gözeneklilik (Porosity)',
    3: 'NoDifetto - Hatasız (No Defect)'
}

print("✅ Hazır!")

✅ Tüm kütüphaneler yüklendi!
🔧 Kullanılan cihaz: cuda
✅ Hazır!


## ResNet18 Modelini Yükle

In [3]:
print("🔄 ResNet18 modeli yükleniyor...")

# ResNet18 yapısını oluştur
model = models.resnet18(weights=None)
model.fc = nn.Linear(512, 4)  # 4 sınıf için

# Eğitilmiş ağırlıkları yükle
model.load_state_dict(torch.load('resnet18_best.pth', map_location=device))
model = model.to(device)
model.eval()  # Test moduna geç

print("✅ ResNet18 modeli hazır!")
print(f"📊 Model: ResNet18")
print(f"🎯 Test Accuracy: %100.00")
print(f"📉 Test Loss: 0.0002")

🔄 ResNet18 modeli yükleniyor...
✅ ResNet18 modeli hazır!
📊 Model: ResNet18
🎯 Test Accuracy: %100.00
📉 Test Loss: 0.0002


## Görüntü İşleme

In [5]:
# Görüntü dönüşümü (eğitimle aynı)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

def preprocess_image(image):
    """Görüntüyü model için hazırla"""
    # PIL Image'e çevir
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)
    
    # RGB'ye çevir
    image = image.convert('RGB')
    
    # Transform uygula ve batch dimension ekle
    image_tensor = transform(image).unsqueeze(0)
    
    return image_tensor

print("✅ Görüntü işleme hazır!")

✅ Görüntü işleme hazır!


## Tahmin Fonksiyonu

In [6]:
def predict(image):
    """
    Görüntüyü sınıflandır ve sonuçları döndür
    """
    try:
        # Görüntüyü hazırla
        image_tensor = preprocess_image(image).to(device)
        
        # Tahmin yap
        with torch.no_grad():
            outputs = model(image_tensor)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
        
        # Sonuçları hazırla (Gradio için dict formatı)
        probs = probabilities[0].cpu().numpy()
        result = {class_names[i]: float(probs[i]) for i in range(4)}
        
        return result
        
    except Exception as e:
        return {"Hata": f"Bir sorun oluştu: {str(e)}"}

print("✅ Tahmin fonksiyonu hazır!")

✅ Tahmin fonksiyonu hazır!


## Gradio arayüzü

In [14]:
# Gradio arayüzü oluştur
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="📸 Kaynak Görüntüsü Yükleyin"),
    outputs=gr.Label(num_top_classes=4, label="🎯 Tahmin Sonuçları"),
    
    title="🏆 Kaynak Hatası Tespit Sistemi - ResNet18",
    
    description="""
    ### 🤖 %100 Accuracy ile Kaynak Hatası Tespiti
    
    Bu sistem **ResNet18** modelini kullanarak kaynak radyografi görüntülerindeki hataları tespit eder.
    
    **Model Performansı:**
    - ✅ Test Accuracy: **%100.00** (2,443 görüntünün tamamı doğru!)
    - ✅ Transfer Learning ile eğitildi (ImageNet)
    - ✅ 15 epoch'ta eğitildi
    
    **Tespit Edilen Hatalar:**
    - 🔴 **Çatlak (Crack)** - Kaynakta oluşan çatlaklar
    - 🟡 **Nüfuziyet Eksikliği** - Yetersiz kaynak penetrasyonu
    - 🟢 **Gözeneklilik (Porosity)** - Kaynak içindeki boşluklar
    - ⚪ **Hatasız** - Kaliteli, kusursuz kaynak
    
    ---
    
    ### 📌 Nasıl Kullanılır:
    1. Bir kaynak radyografi görüntüsü yükleyin (Sürükle-bırak veya dosya seç)
    2. Model otomatik olarak analiz edecek
    3. Her sınıf için güven skorlarını göreceksiniz
    
    💡 **İpucu:** En iyi sonuçlar için 224x224 veya daha büyük görüntüler kullanın.
    """,
    
    article="""
    ---
    
    ### 📊 Proje Detayları:
    
    **Veri Seti:**
    - İsim: RIAWELC (Radiographic Images for Automatic Weld Defects Classification)
    - Toplam: 24,407 görüntü (224x224 piksel)
    - Eğitim: 15,863 | Validation: 6,101 | Test: 2,443
    
    **Model:**
    - Mimari: ResNet18 (Transfer Learning)
    - Framework: PyTorch
    - Eğitim Süresi: ~1 saat (GPU)
    - Test Accuracy: %100.00 🏆
    
    **Geliştirici:** Emine BALYAN ÇELİKKAYA
    **Kaynak:** [RIAWELC Dataset](https://github.com/stefyste/RIAWELC)  
    **Citation:** Benito Totino, Fanny Spagnolo, Stefania Perri (2022)
    
    ---
    
    🎓 Bu proje deep learning, computer vision ve transfer learning tekniklerini kullanarak geliştirilmiştir.
    """,
    
    theme=gr.themes.Soft(),
    flagging_mode="never"
)

print("✅ Gradio arayüzü oluşturuldu!")

C:\Users\Hasan\anaconda3\envs\weld_app\lib\site-packages\gradio\interface.py:171: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  super().__init__(


✅ Gradio arayüzü oluşturuldu!


## Arayüzü Baslat 

In [12]:
print("="*70)
print("🌐 GRADIO BAŞLATILIYOR...")
print("="*70)
print("\n✅ Arayüz hazır!")
print("👉 Şu linke tıkla: http://127.0.0.1:7861\n")

demo.launch(
    share=False,
    inbrowser=True,
 
)

🌐 GRADIO BAŞLATILIYOR...

✅ Arayüz hazır!
👉 Şu linke tıkla: http://127.0.0.1:7861

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
